# 🕸️ LangGraph: Quando o Agente Precisa de um GPS!

**Módulo 14/15 - Curso LangChain v0.2**

E aí, galera! Chegamos ao Módulo 14 e agora vamos falar de uma parada que é o **próximo nível** dos agents que já vimos!

Lembra dos agents que criamos no Módulo 9? Pois é, eles eram legais, mas meio... como posso dizer... **desorganizados**! Era tipo aquele amigo que sai de casa sem GPS e fica rodando pela cidade sem rumo.

**E o LangGraph?** É justamente o GPS dos nossos agents! 🚗📍

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_01.png)

## 🤔 Tá, mas o que é o LangGraph afinal?

Vou te explicar com uma analogia brasileira que você vai entender na hora!

Imagina que você está organizando um churrasco (eu sei, toda explicação de tech no Brasil tem que ter churrasco 😄):

**Agents tradicionais** = Você sozinho fazendo tudo: comprando carne, acendendo churrasqueira, fazendo farofa... É muito trabalho e você pode se perder no meio!

**LangGraph** = Você com uma equipe organizada: João cuida da carne, Maria faz a farofa, Pedro organiza as bebidas, e **você coordena todo mundo** seguindo um plano bem definido!

### O que o LangGraph faz?

- **Cria fluxos complexos** de decisão para agents
- **Gerencia estados** entre diferentes etapas  
- **Permite loops e condições** (coisa que agent simples não faz direito)
- **Controla a execução** de múltiplos agents trabalhando juntos
- **Oferece observabilidade** total do que está rolando

### Por que precisamos dele?

Lembra dos agents que criamos? Eles eram **lineares**: pergunta → ferramenta → resposta. 

Mas na vida real, precisamos de fluxos tipo:
- "Se isso, então aquilo"
- "Tenta de novo se der erro"
- "Faz essa parte, depois volta e faz aquela"
- "Vários agents trabalhando em paralelo"

É aí que entra o **LangGraph**! 🎯

In [ ]:
# Primeiro, vamos instalar as dependências que vamos usar
# Bora configurar nosso ambiente!

!pip install -q langgraph langchain-google-genai python-dotenv matplotlib networkx

# Imports básicos para começar nossa jornada
import os
import json
from typing import Dict, Any, List
from dotenv import load_dotenv

# LangGraph imports - as estrelas do show!
from langgraph.graph import Graph, StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# Para visualizar nossos grafos
import matplotlib.pyplot as plt
import networkx as nx

print("📦 Bibliotecas instaladas e importadas!")
print("🚀 Bora mergulhar no mundo do LangGraph!")

In [ ]:
# Configuração da API - igual fazíamos antes!
load_dotenv()

# Nosso modelo que já conhecemos bem
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0.7
)

print("🤖 Modelo configurado!")
print(f"✅ Usando: {llm.model}")

# Teste rápido para garantir que está funcionando
response = llm.invoke("Olá! Estou aprendendo LangGraph hoje!")
print(f"\n🎯 Teste: {response.content[:100]}...")

## 📊 Conceitos Fundamentais do LangGraph

Antes de botar a mão na massa, vamos entender os conceitos principais. É como aprender as regras do futebol antes de entrar em campo!

### 1. **Graph (Grafo)**
É a estrutura principal - pensa nele como o **mapa do nosso churrasco**. Define quem faz o quê e em que ordem.

### 2. **Nodes (Nós)**
São as **funções ou agents** que fazem o trabalho. Cada nó é como uma pessoa no churrasco:
- Nó "comprador" → vai no açougue
- Nó "churrasqueiro" → cuida da carne
- Nó "organizador" → coordena tudo

### 3. **Edges (Arestas)**
São as **conexões** entre os nós. Definem o fluxo: "depois de comprar a carne, acenda a churrasqueira".

### 4. **State (Estado)**
É a **memória compartilhada** do grafo. Todo mundo no churrasco sabe: "quantas pessoas vão vir?", "que horas começar?", etc.

### 5. **Conditional Edges (Arestas Condicionais)**
O **if/else** do grafo: "Se está chovendo, faz o churrasco na garagem, senão faz no quintal".

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_02.png)

## 🧮 A Matemática por trás do LangGraph

Vou te mostrar a teoria sem assustar! 😅

Um grafo no LangGraph pode ser representado matematicamente como:

$$G = (V, E, S, F)$$

Onde:
- $V$ = conjunto de vértices (nossos nós/funções)
- $E$ = conjunto de arestas (conexões entre nós)
- $S$ = estado compartilhado (nossa memória)
- $F$ = função de transição de estado

### Função de Transição de Estado

Cada nó executa uma função que pode ser representada como:

$$S_{novo} = f_i(S_{atual}, Input_i)$$

Onde:
- $f_i$ é a função do nó $i$
- $S_{atual}$ é o estado antes da execução
- $Input_i$ é a entrada específica do nó
- $S_{novo}$ é o estado resultante

### Algoritmo de Execução

O LangGraph usa um algoritmo similar ao **BFS (Breadth-First Search)** com controle de estado:

1. **Inicializa** estado $S_0$
2. **Executa** nó inicial: $S_1 = f_{start}(S_0)$
3. **Avalia** condições de transição
4. **Seleciona** próximo nó baseado em $S_{atual}$
5. **Repete** até atingir condição de parada

**Dica do Pedro**: Não se preocupe muito com a matemática agora. O importante é entender que o LangGraph gerencia tudo isso para você! É como usar GPS - você não precisa saber os algoritmos de roteamento para chegar no destino! 🗺️

In [ ]:
# Vamos criar nosso primeiro grafo simples!
# Começando com o básico: um contador que toma decisões

from typing import TypedDict

# Definindo o estado do nosso grafo
# É como definir "que informações todo mundo vai compartilhar"
class CounterState(TypedDict):
    count: int
    messages: List[str]
    should_continue: bool

# Função que incrementa o contador
def increment_counter(state: CounterState) -> CounterState:
    """Incrementa o contador e adiciona uma mensagem"""
    new_count = state["count"] + 1
    new_message = f"Contador agora está em: {new_count}"
    
    return {
        "count": new_count,
        "messages": state["messages"] + [new_message],
        "should_continue": new_count < 5  # Para quando chegar em 5
    }

# Função que decide se devemos continuar
def should_continue_counting(state: CounterState) -> str:
    """Decide se devemos continuar ou parar"""
    if state["should_continue"]:
        return "continue"
    else:
        return "stop"

# Função final
def finish_counting(state: CounterState) -> CounterState:
    """Finaliza a contagem"""
    return {
        **state,
        "messages": state["messages"] + ["🎉 Contagem finalizada!"]
    }

print("🔧 Funções do grafo criadas!")
print("📝 Estado definido: count, messages, should_continue")

In [ ]:
# Agora vamos montar nosso grafo!
# É como montar um fluxograma, mas em código

from langgraph.graph import StateGraph, END

# Criando o grafo
workflow = StateGraph(CounterState)

# Adicionando os nós (as "estações" do nosso fluxo)
workflow.add_node("increment", increment_counter)
workflow.add_node("finish", finish_counting)

# Definindo o ponto de entrada
workflow.set_entry_point("increment")

# Adicionando as conexões condicionais
# É aqui que a mágica acontece!
workflow.add_conditional_edges(
    "increment",  # De qual nó
    should_continue_counting,  # Função que decide
    {
        "continue": "increment",  # Se continuar, volta pro increment
        "stop": "finish"         # Se parar, vai pro finish
    }
)

# Conectando o nó final ao fim do grafo
workflow.add_edge("finish", END)

# Compilando o grafo (transformando em algo executável)
app = workflow.compile()

print("🏗️ Grafo construído e compilado!")
print("🎯 Pronto para executar nosso primeiro fluxo!")

In [ ]:
# Hora de executar nosso grafo!
# É como apertar o play na nossa máquina de estados

# Estado inicial
initial_state = {
    "count": 0,
    "messages": ["🚀 Iniciando contagem..."],
    "should_continue": True
}

print("▶️ Executando o grafo...\n")

# Executando o grafo
result = app.invoke(initial_state)

# Mostrando os resultados
print("📊 Resultado Final:")
print(f"Contador final: {result['count']}")
print(f"Deve continuar: {result['should_continue']}")
print("\n📝 Histórico de mensagens:")
for i, msg in enumerate(result['messages'], 1):
    print(f"{i}. {msg}")

print("\n🎉 Liiindo! Nosso primeiro grafo funcionou!")

## 📈 Visualizando nosso Grafo

Uma imagem vale mais que mil palavras! Vamos ver como nosso grafo fica visualmente.

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_03.png)

In [ ]:
# Vamos criar uma visualização do nosso grafo
# Porque ver é melhor que imaginar!

import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.patches import FancyBboxPatch

# Criando o gráfico
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 8)
ax.axis('off')

# Definindo posições dos elementos
positions = {
    'start': (1, 4),
    'increment': (3, 4),
    'decision': (5, 4),
    'finish': (7, 4),
    'end': (9, 4)
}

# Desenhando os nós
for name, (x, y) in positions.items():
    if name == 'decision':
        # Losango para decisão
        diamond = FancyBboxPatch(
            (x-0.5, y-0.3), 1, 0.6,
            boxstyle="round,pad=0.1",
            facecolor='lightblue',
            edgecolor='blue',
            transform=ax.transData
        )
        ax.add_patch(diamond)
        ax.text(x, y, 'Continuar?', ha='center', va='center', fontsize=10, weight='bold')
    else:
        # Retângulo para ações
        rect = FancyBboxPatch(
            (x-0.5, y-0.3), 1, 0.6,
            boxstyle="round,pad=0.1",
            facecolor='lightgreen',
            edgecolor='darkgreen',
            transform=ax.transData
        )
        ax.add_patch(rect)
        ax.text(x, y, name.title(), ha='center', va='center', fontsize=10, weight='bold')

# Desenhando as setas
arrows = [
    ((1.5, 4), (2.5, 4), 'Entrada'),
    ((3.5, 4), (4.5, 4), ''),
    ((5.5, 4), (6.5, 4), 'Não'),
    ((7.5, 4), (8.5, 4), ''),
    ((5, 4.3), (3, 4.7), 'Sim'),  # Loop de volta
]

for (x1, y1), (x2, y2), label in arrows:
    if label == 'Sim':  # Curva para o loop
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', lw=2, color='red',
                                 connectionstyle="arc3,rad=0.3"))
        ax.text(4, 5, label, ha='center', va='center', fontsize=9, color='red', weight='bold')
    else:
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))
        if label:
            mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
            ax.text(mid_x, mid_y + 0.2, label, ha='center', va='center', 
                   fontsize=9, color='darkblue', weight='bold')

ax.set_title('🕸️ Estrutura do nosso LangGraph - Contador', fontsize=16, weight='bold', pad=20)

plt.tight_layout()
plt.show()

print("📊 Visualização criada!")
print("🎯 Observe como o fluxo funciona: Start → Increment → Decisão → Finish ou Loop")

## 🤖 LangGraph com LLMs: Agora fica sério!

Tá, contador é legal, mas vamos fazer algo mais parecido com o que usamos no mundo real!

Vamos criar um grafo que usa nosso modelo LLM para:
1. **Analisar** uma pergunta do usuário
2. **Decidir** se precisa de mais informações
3. **Fazer perguntas** de esclarecimento se necessário
4. **Gerar** a resposta final

É como ter um assistente que não só responde, mas **pensa no processo** de como responder melhor!

### Casos de uso reais:
- **Chatbots inteligentes** que fazem perguntas de esclarecimento
- **Sistemas de suporte** que coletam informações antes de ajudar
- **Assistentes de vendas** que qualificam leads
- **Tutores virtuais** que adaptam explicações

**Dica do Pedro**: Lembra dos agents do Módulo 9? Eles eram ótimos para tarefas simples, mas aqui conseguimos criar **fluxos de conversação muito mais sofisticados**! 🧠

In [ ]:
# Vamos criar um assistente inteligente que pensa antes de responder!
# Estado mais complexo para nosso chatbot

from langchain_core.messages import BaseMessage
from typing import Annotated

class ChatState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    user_question: str
    needs_clarification: bool
    clarification_asked: bool
    context_info: Dict[str, Any]
    final_answer: str

# Nó 1: Analisa se a pergunta precisa de esclarecimento
def analyze_question(state: ChatState) -> ChatState:
    """Analisa se a pergunta do usuário precisa de mais informações"""
    
    question = state["user_question"]
    
    analysis_prompt = f"""
    Analise esta pergunta do usuário: "{question}"
    
    Determine se a pergunta é:
    - CLARA: pode ser respondida diretamente
    - AMBÍGUA: precisa de esclarecimentos
    
    Responda apenas com: CLARA ou AMBÍGUA
    """
    
    response = llm.invoke(analysis_prompt)
    needs_clarification = "AMBÍGUA" in response.content.upper()
    
    return {
        **state,
        "needs_clarification": needs_clarification,
        "messages": [AIMessage(content=f"Análise: {'Precisa esclarecimento' if needs_clarification else 'Pergunta clara'}")]
    }

# Nó 2: Faz pergunta de esclarecimento
def ask_clarification(state: ChatState) -> ChatState:
    """Gera uma pergunta de esclarecimento para o usuário"""
    
    question = state["user_question"]
    
    clarification_prompt = f"""
    O usuário fez esta pergunta ambígua: "{question}"
    
    Faça UMA pergunta específica para esclarecer o que ele realmente quer saber.
    Seja direto e útil.
    """
    
    response = llm.invoke(clarification_prompt)
    
    return {
        **state,
        "clarification_asked": True,
        "messages": state["messages"] + [AIMessage(content=response.content)]
    }

# Nó 3: Gera resposta final
def generate_final_answer(state: ChatState) -> ChatState:
    """Gera a resposta final baseada em todas as informações"""
    
    question = state["user_question"]
    context = "\n".join([msg.content for msg in state["messages"] if msg.content])
    
    final_prompt = f"""
    Pergunta original: "{question}"
    Contexto da conversa: {context}
    
    Forneça uma resposta completa e útil para o usuário.
    Seja claro, direto e amigável.
    """
    
    response = llm.invoke(final_prompt)
    
    return {
        **state,
        "final_answer": response.content,
        "messages": state["messages"] + [AIMessage(content=response.content)]
    }

print("🧠 Nós do chatbot inteligente criados!")
print("✅ Funções: analyze_question, ask_clarification, generate_final_answer")

In [ ]:
# Função de decisão para nosso chatbot
def should_ask_clarification(state: ChatState) -> str:
    """Decide o próximo passo baseado no estado atual"""
    
    # Se precisa de esclarecimento e ainda não perguntou
    if state["needs_clarification"] and not state["clarification_asked"]:
        return "ask_clarification"
    
    # Senão, gera a resposta final
    return "generate_answer"

# Construindo nosso chatbot inteligente
chatbot_workflow = StateGraph(ChatState)

# Adicionando os nós
chatbot_workflow.add_node("analyze", analyze_question)
chatbot_workflow.add_node("clarify", ask_clarification)
chatbot_workflow.add_node("answer", generate_final_answer)

# Definindo o fluxo
chatbot_workflow.set_entry_point("analyze")

# Conexões condicionais
chatbot_workflow.add_conditional_edges(
    "analyze",
    should_ask_clarification,
    {
        "ask_clarification": "clarify",
        "generate_answer": "answer"
    }
)

# Da clarificação sempre vai para a resposta
chatbot_workflow.add_edge("clarify", "answer")

# Resposta final termina o fluxo
chatbot_workflow.add_edge("answer", END)

# Compilando nosso chatbot
chatbot_app = chatbot_workflow.compile()

print("🤖 Chatbot inteligente montado!")
print("🎯 Fluxo: Analisa → Decide → Esclarece (se necessário) → Responde")

In [ ]:
# Testando nosso chatbot inteligente!
# Vamos ver como ele se comporta com diferentes tipos de pergunta

def test_chatbot(question: str):
    """Testa o chatbot com uma pergunta"""
    
    print(f"\n🗣️ Usuário: {question}")
    print("="*50)
    
    # Estado inicial
    initial_state = {
        "messages": [],
        "user_question": question,
        "needs_clarification": False,
        "clarification_asked": False,
        "context_info": {},
        "final_answer": ""
    }
    
    # Executando
    result = chatbot_app.invoke(initial_state)
    
    # Mostrando o processo
    print("🔄 Processo:")
    for i, msg in enumerate(result["messages"], 1):
        print(f"{i}. {msg.content}")
    
    print(f"\n🎯 Precisou esclarecimento: {'Sim' if result['needs_clarification'] else 'Não'}")
    
    return result

# Teste 1: Pergunta clara
print("🧪 TESTE 1: Pergunta Clara")
test1 = test_chatbot("Qual é a capital do Brasil?")

# Teste 2: Pergunta ambígua
print("\n\n🧪 TESTE 2: Pergunta Ambígua")
test2 = test_chatbot("Como faço isso?")

print("\n🎉 Testes concluídos! Veja como o chatbot se adapta ao tipo de pergunta!")

## 📊 Comparação: Agents vs LangGraph

Agora que vimos os dois em ação, vamos comparar!

| Aspecto | Agents (Módulo 9) | LangGraph |
|---------|-------------------|----------|
| **Complexidade** | Simples, linear | Complexo, ramificado |
| **Controle de Fluxo** | Limitado | Total |
| **Estados** | Não gerencia | Gerencia completamente |
| **Loops** | Difícil | Nativo |
| **Múltiplos Caminhos** | Não suporta | Suporta nativamente |
| **Debugging** | Difícil | Fácil (observável) |
| **Casos de Uso** | Tarefas simples | Workflows complexos |

### Quando usar cada um?

**Use Agents quando**:
- Tarefa simples e direta
- Não precisa de loops ou condições
- Quer algo rápido de implementar

**Use LangGraph quando**:
- Precisa de fluxos condicionais
- Quer controlar cada etapa
- Precisa de loops ou retry logic
- Quer observabilidade total
- Múltiplos agents trabalhando juntos

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_04.png)

In [ ]:
# Vamos criar um gráfico comparativo de performance
# Simulando cenários diferentes

import numpy as np
import matplotlib.pyplot as plt

# Dados simulados de performance
scenarios = ['Tarefa Simples', 'Fluxo Condicional', 'Múltiplos Loops', 'Error Handling', 'Observabilidade']
agents_score = [9, 4, 2, 3, 2]  # Agents tradicionais
langgraph_score = [7, 9, 9, 8, 10]  # LangGraph

x = np.arange(len(scenarios))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 7))
rects1 = ax.bar(x - width/2, agents_score, width, label='Agents Tradicionais', 
                color='lightcoral', alpha=0.8)
rects2 = ax.bar(x + width/2, langgraph_score, width, label='LangGraph', 
                color='lightblue', alpha=0.8)

ax.set_ylabel('Score (0-10)', fontsize=12)
ax.set_title('🏆 Agents vs LangGraph - Comparação de Capabilities', fontsize=14, weight='bold')
ax.set_xticks(x)
ax.set_xticklabels(scenarios, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Adicionando valores nas barras
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', weight='bold')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

# Análise dos resultados
print("📊 Análise dos Resultados:")
print("\n🟢 Agents Tradicionais são melhores em:")
for i, scenario in enumerate(scenarios):
    if agents_score[i] > langgraph_score[i]:
        print(f"   • {scenario} (Score: {agents_score[i]} vs {langgraph_score[i]})")

print("\n🔵 LangGraph é melhor em:")
for i, scenario in enumerate(scenarios):
    if langgraph_score[i] > agents_score[i]:
        print(f"   • {scenario} (Score: {langgraph_score[i]} vs {agents_score[i]})")

print("\n🎯 Conclusão: LangGraph vence em cenários complexos, Agents em tarefas simples!")

## 🏗️ Projeto Prático: Sistema de Atendimento Inteligente

Bora fazer um projeto real que você pode usar no trabalho!

Vamos criar um **Sistema de Atendimento ao Cliente** que:

1. **Classifica** o tipo de problema
2. **Coleta** informações necessárias
3. **Tenta resolver** automaticamente
4. **Escalona** para humano se necessário

É tipo um **funil inteligente** que otimiza o atendimento!

### Fluxo do Sistema:
```mermaid
graph TD
    A[Cliente faz pergunta] --> B[Classifica problema]
    B --> C{Tipo de problema?}
    C -->|Técnico| D[Coleta info técnica]
    C -->|Comercial| E[Coleta info comercial]
    C -->|Suporte| F[Coleta info suporte]
    D --> G[Tenta resolver]
    E --> G
    F --> G
    G --> H{Conseguiu resolver?}
    H -->|Sim| I[Resposta final]
    H -->|Não| J[Escalona para humano]
```

**Dica do Pedro**: Este é o tipo de sistema que grandes empresas pagam milhões para ter! E você vai aprender a fazer em algumas células! 💪

In [ ]:
# Estado do nosso sistema de atendimento
class SupportState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    customer_query: str
    problem_type: str  # 'technical', 'commercial', 'support'
    collected_info: Dict[str, Any]
    resolution_attempt: str
    resolved: bool
    escalated: bool
    final_response: str

# Nó 1: Classifica o tipo de problema
def classify_problem(state: SupportState) -> SupportState:
    """Classifica o tipo de problema do cliente"""
    
    query = state["customer_query"]
    
    classification_prompt = f"""
    Classifique este problema do cliente em uma das categorias:
    
    Problema: "{query}"
    
    Categorias:
    - TECHNICAL: problemas técnicos, bugs, não funciona
    - COMMERCIAL: preços, planos, vendas, upgrades
    - SUPPORT: como usar, tutoriais, dúvidas gerais
    
    Responda apenas com: TECHNICAL, COMMERCIAL ou SUPPORT
    """
    
    response = llm.invoke(classification_prompt)
    problem_type = response.content.strip().upper()
    
    if problem_type not in ['TECHNICAL', 'COMMERCIAL', 'SUPPORT']:
        problem_type = 'SUPPORT'  # Default
    
    return {
        **state,
        "problem_type": problem_type.lower(),
        "messages": [AIMessage(content=f"Problema classificado como: {problem_type}")]
    }

# Nó 2: Coleta informações específicas
def collect_info(state: SupportState) -> SupportState:
    """Coleta informações baseadas no tipo de problema"""
    
    problem_type = state["problem_type"]
    query = state["customer_query"]
    
    if problem_type == "technical":
        info_prompt = f"""
        Para resolver este problema técnico: "{query}"
        
        Que informações técnicas você precisa coletar?
        Liste 3 perguntas específicas que ajudariam a diagnosticar.
        """
    elif problem_type == "commercial":
        info_prompt = f"""
        Para esta questão comercial: "{query}"
        
        Que informações comerciais você precisa?
        Liste 3 perguntas sobre necessidades e orçamento.
        """
    else:  # support
        info_prompt = f"""
        Para esta dúvida de suporte: "{query}"
        
        Que esclarecimentos você precisa?
        Liste 3 perguntas para entender melhor a necessidade.
        """
    
    response = llm.invoke(info_prompt)
    
    return {
        **state,
        "collected_info": {"questions": response.content},
        "messages": state["messages"] + [AIMessage(content=f"Informações coletadas para {problem_type}")]
    }

print("🎯 Funções de classificação e coleta criadas!")

In [ ]:
# Nó 3: Tenta resolver o problema
def attempt_resolution(state: SupportState) -> SupportState:
    """Tenta resolver o problema automaticamente"""
    
    query = state["customer_query"]
    problem_type = state["problem_type"]
    info = state["collected_info"]
    
    resolution_prompt = f"""
    Tente resolver este problema do cliente:
    
    Problema: "{query}"
    Tipo: {problem_type}
    Informações coletadas: {info}
    
    Forneça uma solução detalhada e prática.
    Se não conseguir resolver completamente, diga "ESCALATE".
    """
    
    response = llm.invoke(resolution_prompt)
    
    # Verifica se conseguiu resolver
    resolved = "ESCALATE" not in response.content.upper()
    
    return {
        **state,
        "resolution_attempt": response.content,
        "resolved": resolved,
        "messages": state["messages"] + [AIMessage(content="Tentativa de resolução realizada")]
    }

# Nó 4: Resposta final (quando resolve)
def provide_final_answer(state: SupportState) -> SupportState:
    """Fornece a resposta final quando o problema foi resolvido"""
    
    resolution = state["resolution_attempt"]
    
    final_prompt = f"""
    Transforme esta solução técnica em uma resposta amigável para o cliente:
    
    Solução: {resolution}
    
    Seja caloroso, claro e útil. Termine perguntando se precisa de mais alguma coisa.
    """
    
    response = llm.invoke(final_prompt)
    
    return {
        **state,
        "final_response": response.content,
        "messages": state["messages"] + [AIMessage(content=response.content)]
    }

# Nó 5: Escalona para humano
def escalate_to_human(state: SupportState) -> SupportState:
    """Escalona o problema para atendimento humano"""
    
    query = state["customer_query"]
    problem_type = state["problem_type"]
    
    escalation_msg = f"""
    🤝 Olá! Vou conectar você com um de nossos especialistas.
    
    Seu problema ({problem_type}) será tratado com prioridade.
    Tempo estimado de espera: 5-10 minutos.
    
    Um momento, por favor...
    """
    
    return {
        **state,
        "escalated": True,
        "final_response": escalation_msg,
        "messages": state["messages"] + [AIMessage(content=escalation_msg)]
    }

print("🔧 Funções de resolução e escalonamento criadas!")
print("✅ Sistema de atendimento quase pronto!")

In [ ]:
# Função de decisão do sistema
def decide_next_step(state: SupportState) -> str:
    """Decide se resolve ou escalona"""
    
    if state["resolved"]:
        return "provide_answer"
    else:
        return "escalate"

# Montando o sistema completo
support_workflow = StateGraph(SupportState)

# Adicionando todos os nós
support_workflow.add_node("classify", classify_problem)
support_workflow.add_node("collect", collect_info)
support_workflow.add_node("resolve", attempt_resolution)
support_workflow.add_node("answer", provide_final_answer)
support_workflow.add_node("escalate", escalate_to_human)

# Definindo o fluxo
support_workflow.set_entry_point("classify")

# Conexões lineares até a decisão
support_workflow.add_edge("classify", "collect")
support_workflow.add_edge("collect", "resolve")

# Decisão condicional após tentar resolver
support_workflow.add_conditional_edges(
    "resolve",
    decide_next_step,
    {
        "provide_answer": "answer",
        "escalate": "escalate"
    }
)

# Ambos os finais terminam o fluxo
support_workflow.add_edge("answer", END)
support_workflow.add_edge("escalate", END)

# Compilando nosso sistema de atendimento
support_app = support_workflow.compile()

print("🏢 Sistema de Atendimento Inteligente PRONTO!")
print("🎯 Fluxo completo montado e testado!")

In [ ]:
# Testando nosso sistema de atendimento!
# Vamos simular diferentes tipos de problemas

def test_support_system(customer_query: str):
    """Testa o sistema de atendimento"""
    
    print(f"\n🎧 NOVO ATENDIMENTO")
    print(f"Cliente: {customer_query}")
    print("="*60)
    
    # Estado inicial
    initial_state = {
        "messages": [],
        "customer_query": customer_query,
        "problem_type": "",
        "collected_info": {},
        "resolution_attempt": "",
        "resolved": False,
        "escalated": False,
        "final_response": ""
    }
    
    # Executando o sistema
    result = support_app.invoke(initial_state)
    
    # Relatório do atendimento
    print(f"\n📊 RELATÓRIO DO ATENDIMENTO:")
    print(f"Tipo do problema: {result['problem_type'].upper()}")
    print(f"Resolvido automaticamente: {'✅ Sim' if result['resolved'] else '❌ Não'}")
    print(f"Escalonado: {'✅ Sim' if result['escalated'] else '❌ Não'}")
    
    print(f"\n💬 RESPOSTA FINAL:")
    print(result['final_response'])
    
    return result

# Teste 1: Problema técnico simples
print("🧪 TESTE 1: Problema Técnico")
test1 = test_support_system("Meu aplicativo não está abrindo no celular")

# Teste 2: Dúvida comercial
print("\n\n🧪 TESTE 2: Questão Comercial")
test2 = test_support_system("Quero saber sobre os planos premium")

# Teste 3: Problema complexo (deve escalonar)
print("\n\n🧪 TESTE 3: Problema Complexo")
test3 = test_support_system("Perdi todos os meus dados após a atualização")

print("\n🎉 Sistema testado! Veja como ele se adapta a diferentes situações!")

## 🎯 Exercício Prático: Seu Primeiro LangGraph

Agora é sua vez de botar a mão na massa!

### 🏆 DESAFIO 1: Sistema de Recomendação Inteligente

Crie um LangGraph que:
1. **Pergunta** sobre os gostos do usuário
2. **Analisa** as preferências
3. **Decide** se tem informação suficiente
4. **Faz mais perguntas** se necessário
5. **Gera** recomendações personalizadas

#### Requisitos:
- Use pelo menos 4 nós
- Implemente pelo menos 1 condicional
- Gerencie um estado com múltiplas variáveis
- Teste com diferentes cenários

#### Dica do Pedro:
Comece simples! Defina o estado, crie as funções, monte o grafo e teste. Depois você pode incrementar! 🚀

In [ ]:
# SEU CÓDIGO AQUI! 🚀
# Implemente o sistema de recomendação

# 1. Defina o estado (RecommendationState)
# 2. Crie as funções dos nós
# 3. Monte o grafo
# 4. Teste!

# Estrutura sugerida:
# class RecommendationState(TypedDict):
#     user_preferences: Dict[str, Any]
#     questions_asked: int
#     enough_info: bool
#     recommendations: List[str]
#     # ... outros campos que você achar necessário

# Dica: Comece definindo que tipo de recomendação vai fazer
# (filmes, restaurantes, livros, etc.)

print("💪 Seu desafio está aqui!")
print("🎯 Implemente um sistema de recomendação com LangGraph!")

# APAGUE ESTE COMENTÁRIO E IMPLEMENTE SUA SOLUÇÃO

## 🏅 Exercício Avançado: Multi-Agent System

### 🚀 DESAFIO 2: Sistema de Análise de Texto Multi-Agente

Para quem quer ir além! Crie um sistema onde **múltiplos especialistas** analisam um texto:

1. **Agent Análise Sentimento** - Analisa se é positivo/negativo
2. **Agent Extração Tópicos** - Identifica temas principais  
3. **Agent Resumo** - Cria um resumo
4. **Agent Coordenador** - Combina todos os resultados

#### Fluxo:
```mermaid
graph TD
    A[Texto de entrada] --> B[Análise Sentimento]
    A --> C[Extração Tópicos]
    A --> D[Resumo]
    B --> E[Coordenador]
    C --> E
    D --> E
    E --> F[Relatório Final]
```

#### Requisitos Avançados:
- Processamento em paralelo (simule com sleep)
- Agregação inteligente dos resultados
- Sistema de pontuação de confiança
- Tratamento de erros

**Dica do Pedro**: Este é nível **ninja**! Se conseguir fazer, você já entendeu o poder real do LangGraph! 🥷

In [ ]:
# DESAFIO NINJA! 🥷
# Sistema Multi-Agent de Análise de Texto

# Sua missão (se aceitar):
# 1. Criar múltiplos agents especializados
# 2. Fazer eles trabalharem em paralelo
# 3. Coordenar os resultados
# 4. Gerar um relatório unificado

# Estado sugerido:
# class MultiAnalysisState(TypedDict):
#     input_text: str
#     sentiment_analysis: Dict[str, Any]
#     topic_extraction: Dict[str, Any]
#     summary: str
#     confidence_scores: Dict[str, float]
#     final_report: str

# Dica: Use time.sleep() para simular processamento em paralelo
# Dica 2: Cada agent pode ter sua própria função de "confiança"

import time
import random

print("🥷 DESAFIO NINJA ATIVADO!")
print("🎯 Crie um sistema multi-agent para análise de texto!")
print("💪 Boa sorte, padawan!")

# SEU CÓDIGO AQUI!

## 🔮 O Futuro do LangGraph e próximos passos

Bora falar sobre onde isso tudo vai dar e como você pode evoluir!

### 🚀 Próximas evoluções do LangGraph:

1. **Paralelização Nativa** - Execução real em paralelo
2. **Persistência de Estado** - Estados que sobrevivem entre execuções
3. **Streaming Real-time** - Execução em tempo real
4. **Auto-otimização** - Grafos que se otimizam sozinhos
5. **Integração com Deploy** - Deploy direto na nuvem

### 💼 Casos de uso empresariais reais:

- **Customer Journey Automation** - Jornadas personalizadas
- **Content Moderation** - Moderação inteligente em múltiplas etapas  
- **Sales Qualification** - Qualificação automática de leads
- **Technical Support** - Suporte técnico multiníveis
- **Data Processing Pipelines** - Pipelines de dados inteligentes

### 🎯 Como continuar estudando:

1. **Pratique os exemplos** deste notebook
2. **Crie seus próprios grafos** para problemas reais
3. **Combine com o que aprendeu** nos módulos anteriores
4. **Experimente com diferentes LLMs**
5. **Teste em produção** (próximo módulo: LangSmith!)

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_05.png)

**Dica do Pedro**: LangGraph não é só uma ferramenta, é uma **nova forma de pensar** em IA! Em vez de "o que minha IA faz?", pergunte "como minha IA pensa?". Essa mudança de mindset é revolucionária! 🧠✨

In [ ]:
# Vamos criar uma visualização do "futuro" do LangGraph
# Mostrando a evolução de complexidade que conseguimos atingir

import matplotlib.pyplot as plt
import numpy as np

# Dados da evolução de capacidades
technologies = ['Scripts\nSimples', 'Chains\nLangChain', 'Agents\nTradicionais', 'LangGraph\nBásico', 'LangGraph\nAvançado', 'Multi-Agent\nSystems']
complexity_score = [1, 3, 5, 7, 8, 10]
capabilities = [1, 4, 6, 8, 9, 10]
maintainability = [8, 6, 4, 7, 8, 9]

x = np.arange(len(technologies))
width = 0.25

fig, ax = plt.subplots(figsize=(14, 8))

# Barras para cada métrica
rects1 = ax.bar(x - width, complexity_score, width, label='Complexidade de Problemas', 
                color='#FF6B6B', alpha=0.8)
rects2 = ax.bar(x, capabilities, width, label='Capacidades', 
                color='#4ECDC4', alpha=0.8)
rects3 = ax.bar(x + width, maintainability, width, label='Manutenibilidade', 
                color='#45B7D1', alpha=0.8)

# Configuração do gráfico
ax.set_xlabel('Tecnologias', fontsize=12, weight='bold')
ax.set_ylabel('Score (1-10)', fontsize=12, weight='bold')
ax.set_title('🚀 Evolução das Tecnologias de IA - De Scripts a Multi-Agents', 
             fontsize=14, weight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(technologies, rotation=0, ha='center')
ax.legend(loc='upper left')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 11)

# Adicionando anotações especiais
ax.annotate('🎯 Estamos aqui!', 
            xy=(4, 8.5), xytext=(4.5, 9.5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=12, weight='bold', color='red')

ax.annotate('🔮 O futuro!', 
            xy=(5, 10), xytext=(4.2, 10.5),
            arrowprops=dict(arrowstyle='->', color='purple', lw=2),
            fontsize=12, weight='bold', color='purple')

plt.tight_layout()
plt.show()

print("📈 Evolução das capacidades:")
print("🔥 LangGraph representa um salto qualitativo!")
print("💪 Você agora domina uma das tecnologias mais avançadas de IA!")

## 📚 Resumo do Módulo - O que aprendemos?

Caraca, que jornada! Vamos recapitular tudo que vimos:

### ✅ **Conceitos Fundamentais**
- **LangGraph** = GPS para agents inteligentes
- **Grafos dirigidos** para controle de fluxo
- **Estados compartilhados** entre nós
- **Execução condicional** baseada em lógica

### ✅ **Implementações Práticas**
- Sistema de **contador com loops**
- **Chatbot inteligente** com análise contextual
- **Sistema de atendimento** empresarial completo
- **Fluxos condicionais** e decisões automáticas

### ✅ **Comparações e Insights**
- **Agents vs LangGraph** - quando usar cada um
- **Casos de uso reais** em empresas
- **Visualizações** de fluxos e performance
- **Próximos passos** na evolução da tecnologia

### 🎯 **Principais Diferenças do que vimos antes:**

| Módulo Anterior | LangGraph | 
|-----------------|----------|
| Agents lineares (Módulo 9) | **Fluxos complexos e condicionais** |
| Memory isolada (Módulo 5) | **Estado compartilhado inteligente** |
| Chains simples (Módulo 4) | **Workflows sofisticados** |

### 🚀 **Preparação para o Módulo 15:**
No próximo módulo vamos aprender sobre **LangSmith** - a ferramenta que vai nos ajudar a:
- **Monitorar** nossos LangGraphs em produção
- **Debugar** fluxos complexos
- **Otimizar** performance 
- **Analisar** comportamentos

### 💡 **Dica Final do Pedro:**
LangGraph não é só sobre fazer IA mais complexa - é sobre fazer IA **mais inteligente e controlável**. Você agora tem o poder de criar sistemas que **realmente pensam** antes de agir!

E lembra: **toda expertise começa com curiosidade**. Continue experimentando, criando e se divertindo com o que aprendeu! 🎉

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-14_img_06.png)

In [ ]:
# Parabéns! Você chegou ao final do Módulo 14! 🎉
# Vamos criar um "certificado" visual do que você domina agora

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
import matplotlib.patches as mpatches

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Fundo do certificado
cert_bg = FancyBboxPatch(
    (0.5, 1), 9, 8,
    boxstyle="round,pad=0.1",
    facecolor='#f0f8ff',
    edgecolor='#4169e1',
    linewidth=3
)
ax.add_patch(cert_bg)

# Título
ax.text(5, 8.5, '🏆 CERTIFICADO DE CONCLUSÃO', 
        ha='center', va='center', fontsize=18, weight='bold', color='#4169e1')
ax.text(5, 8, 'Módulo 14: LangGraph Mastery', 
        ha='center', va='center', fontsize=14, weight='bold', color='#2e8b57')

# Linha decorativa
ax.plot([1.5, 8.5], [7.5, 7.5], color='#4169e1', linewidth=2)

# Competências adquiridas
skills = [
    '✅ Criação de Grafos Inteligentes',
    '✅ Gerenciamento de Estados Complexos', 
    '✅ Fluxos Condicionais e Loops',
    '✅ Sistemas Multi-Agent',
    '✅ Integração com LLMs',
    '✅ Debugging e Visualização'
]

y_start = 6.5
for i, skill in enumerate(skills):
    ax.text(1.5, y_start - i*0.4, skill, 
            ha='left', va='center', fontsize=11, color='#2e8b57')

# Estatísticas
ax.text(5, 3.5, '📊 SUAS ESTATÍSTICAS:', 
        ha='center', va='center', fontsize=12, weight='bold', color='#4169e1')

stats = [
    '🧠 Conceitos Aprendidos: 10+',
    '💻 Códigos Executados: 20+', 
    '🏗️ Projetos Criados: 3',
    '🎯 Nível Alcançado: Avançado'
]

y_start = 3
for i, stat in enumerate(stats):
    ax.text(5, y_start - i*0.3, stat, 
            ha='center', va='center', fontsize=10, color='#2e8b57')

# Assinatura do Pedro
ax.text(5, 1.5, '👨‍💻 Pedro Nunes Guth', 
        ha='center', va='center', fontsize=12, weight='bold', color='#4169e1')
ax.text(5, 1.2, 'Instrutor Expert em IA & LangChain', 
        ha='center', va='center', fontsize=10, style='italic', color='#666')

plt.title('🎓 PARABÉNS! VOCÊ DOMINA LANGGRAPH!', 
          fontsize=16, weight='bold', pad=20, color='#4169e1')

plt.tight_layout()
plt.show()

print("🎉 PARABÉNS! Você completou o Módulo 14!")
print("🚀 Agora você domina LangGraph e pode criar sistemas de IA complexos!")
print("📚 Próximo desafio: Módulo 15 - LangSmith para produção!")
print("\n💪 Continue assim, você está arrasando!")